In [85]:
import os 
import json
import pandas as pandas
import traceback
import langchain
from dotenv import load_dotenv
load_dotenv()

True

In [86]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate

In [87]:
OPENAI_KEY=os.getenv("KEY")

In [88]:
print(OPENAI_KEY)

sk-proj-kBC9TwNfdh07E1p83Z7VT3BlbkFJFkodXXR8DuwMYj3dqYw4


In [89]:
llm = ChatOpenAI(openai_api_key = OPENAI_KEY, model_name="gpt-3.5-turbo",
                 temperature=0.7)

c:\Users\Swapnil Jyot\SAE_system\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [90]:
TEMPLATE_gpt = """
You are an expert examiner tasked with evaluating a student's response sheet against the answer key:

The student's response sheet is a Python dictionary represented as {student_dic}, and the answer sheet is {eval_dic}. Each dictionary key corresponds to a question, and its value is the respective answer.

First, calculate the total number of questions by counting the keys in the dictionary.

For each question in the student's response dictionary:
- If it's a THEORY type:
  - **Syntactic Score (0 to 100):** Evaluate grammatical structure and arrangement.
  - **Semantic Score (0 to 100):** Assess relatedness in meaning and conceptual understanding.
  - **Contextual Score (0 to 100):** Consider alignment with specific context or scenario.

- If it's a NUMERICAL type:
  - **First Step Score (0 to 100):** Evaluate correctness of the response's foundation and application of formulas.
  - **Middle Step Score (0 to 100):** Assess intermediate steps towards the final answer.
  - **Final Step Score (0 to 100):** Check if the final response aligns with the answer key.
If you get null value for any particular key then the question response is Not Attempted
Output the evaluation scores in a tabular format like {tabular_response}, ensuring to update each metric strictly for all questions. 
Avoid generating extra tokens to ensure a concise and efficient response.
"""

In [91]:
tabular_response= """[{"Question":"Display that Question",
     "Type":"Numerical",
     "Response":"Attempted",
              "Syntactic Score":null,
              "Semantic Score":null,
              "Contextual Score":null,
              "FirstStep Score":"Score1",
              "Middle Step Score":"Score2",
              "Last Step Score":"Score3",
                "Total Score":"(Score1+Score2+Score3)\/3"},
 {"Question":"Display that Question",
     "Type":"Theory",
     "Response":"Attempted",
              "Syntactic Score":"Score1",
              "Semantic Score":"Score2",
              "Contextual Score":"Score3",
              "FirstStep Score":null,
              "Middle Step Score":null,
              "Last Step Score":null,
                "Total Score":"(Score1+Score2+Score3)\/3"},
 {"Question":"Display that Question",
     "Type":"Theory",
     "Response":"Not Attempted",
               "Syntactic Score":null,
               "Semantic Score":null,
               "Contextual Score":null,
               "FirstStep Score":null,
               "Middle Step Score":null,
               "Last Step Score":null,
                 "Total Score":"(Score1+Score2+Score3)\/3"},
 {"Question":"Display that Question",
     "Type":"Numerical",
     "Response":"Attempted",
              "Syntactic Score":null,
              "Semantic Score":null,"Contextual Score":null,
              "FirstStep Score":"Score1",
              "Middle Step Score":"Score2",
              "Last Step Score":"Score3",
                  "Total Score":"(Score1+Score2+Score3)\/3"},
 {"Question":"Display that Question",
     "Type":"Numerical",
     "Response":"Attempted",
              "Syntactic Score":null,
              "Seantic Score":null,
              "Contextual Score":null,
              "FirstStep Score":"Score1",
              "Middle Step Score":"Score2",
              "Last Step Score":"Score3",
                "Total Score":"(Score1+Score2+Score3)\/3"},
 {"Question Number":"And So on",
     "Type":"Theory",
     "Response":"Attempted",
              "Syntactic Score":"Score1",
              "Semantic Score":"Score2",
              "Contextual Score":"Score3",
              "FirstStep Score":null,
              "Middle Step Score":null,
              "Last Step Score":null,
                "Total Score":"(Score1+Score2+Score3)\/3"}]"""

In [92]:
prompt_template_name=PromptTemplate(
    input_variables=["student_dic","eval_dic","RESPONSE"],
    template=TEMPLATE_gpt
)

In [93]:
first_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="score_1", verbose=True)

c:\Users\Swapnil Jyot\SAE_system\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [94]:
Review_template = """
As an expert in English grammar and writing, please verify that the following score card {score_1} strictly conforms to the specified format, accurately calculating all variables such as total number of questions, type, response, and total score.

Ensure the response remains cost-effective without any additional tokens like headings or alerts, strictly adhering to the required format. If the score card is not in the correct format and contains any headings or alerts, update it and provide the output strictly in the required format:
Don't genereate all these literals , json\n[\n in starting
score cards:
{tabular_response}
"""

In [95]:
review_template_name=PromptTemplate(
    input_variables=["score_1","tabular_response"],
    template=Review_template
)

In [96]:
review_chain = LLMChain(llm=llm, prompt=review_template_name, output_key="Final_score", verbose=True)

In [97]:
score_structure = SequentialChain(
                                chains=[first_chain ,review_chain],
                                input_variables=["student_dic","eval_dic","tabular_response"],
                                output_variables=["score_1","Final_score"],
                                verbose=True
                            )

In [98]:
file_path1 = r"C:\Users\Swapnil Jyot\SAE_system\evaluator.txt"


In [99]:
file_path2=r"C:\Users\Swapnil Jyot\SAE_system\Student.txt"

In [100]:
with open(file_path1,'r') as file:
    evaluator=file.read()

In [101]:
evaluator

'Question: Explain the process of Water cycle?\nAnswer: The water cycle, or hydrological cycle, consists of several stages. Evaporation occurs when\nsurface water transforms into vapor due to heat. Condensation results in cloud formation as water\nvapor cools and condenses into liquid. Precipitation transpires when water droplets or ice crystals\nfall from clouds as rain, snow, sleet, or hail. Runoff entails the surface flow of water back into\noceans, rivers, or lakes.\n\nQuestion: What is GitHub and why is it used?\nAnswer: GitHub is a platform for hosting code that allows for version control and collaboration. \nIt allows you and others to collaborate on projects from anywhere.\nThis lesson will teach you the fundamentals of GitHub, such as repositories, branches, commits, and pull requests..\n\nQuestion:Two resistances are connected in two gaps of Meter Bridge. The balance is 20cm from the zero end.\nA resistance of 15 ohms is connected in series with the smaller of the two.\nThe n

In [102]:
with open(file_path2,'r') as file:
    student=file.read()

In [104]:
import re
import warnings
import pandas as pd

def create_dictionary(content):
    # Remove numeric indices before "Question" and "Answer" words, preserving the colon
    content_without_indices = re.sub(r'(\b\d+\.\s*Question:)', 'Question:', content)
    content_without_indices = re.sub(r'(\b\d+\.\s*Answer:)', 'Answer:', content_without_indices)

    # Tokenize and print each word with punctuation
    words = re.findall(r'\b\w+\b|[.,{};!?:^/−+()=>]', content_without_indices)

    sentence = ""
    my_dic = {}
    n = 0

    for word in words:
        if word == "Answer" and words[n + 1] == ":":
            key = sentence.strip()
            sentence = ""
        if word == "Question" and n > 4 and words[n + 1] == ":":
            my_dic[key] = sentence.strip()
            sentence = ""
        sentence += word
        sentence += " "
        n += 1

    # Check for the last question-answer pair
    if sentence.strip() and key:
        my_dic[key] = sentence.strip()

    return my_dic

In [105]:
eval_dic=create_dictionary( evaluator)

In [106]:
std_dic=create_dictionary(student)

In [107]:
eval_dic

{'Question : Explain the process of Water cycle ?': 'Answer : The water cycle , or hydrological cycle , consists of several stages . Evaporation occurs when surface water transforms into vapor due to heat . Condensation results in cloud formation as water vapor cools and condenses into liquid . Precipitation transpires when water droplets or ice crystals fall from clouds as rain , snow , sleet , or hail . Runoff entails the surface flow of water back into oceans , rivers , or lakes .',
 'Question : What is GitHub and why is it used ?': 'Answer : GitHub is a platform for hosting code that allows for version control and collaboration . It allows you and others to collaborate on projects from anywhere . This lesson will teach you the fundamentals of GitHub , such as repositories , branches , commits , and pull requests . .',
 'Question : Two resistances are connected in two gaps of Meter Bridge . The balance is 20cm from the zero end . A resistance of 15 ohms is connected in series with t

In [108]:
from langchain.callbacks import get_openai_callback

In [115]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=score_structure(
        {
            "student_dic": std_dic,
            "eval_dic": eval_dic,
            "tabular_response": json.dumps(tabular_response),
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert examiner tasked with evaluating a student's response sheet against the answer key:

The student's response sheet is a Python dictionary represented as {'Question : Explain the process of Water cycle ?': 'Answer : A Water cycle is a process underwhich H2O has been evoporates from big water bodies like ocean , seas , river . After that the evoporates steams make cloud of stream droplets when the cloud got dense then the precipitate in the form of rain , hail and snow this is the water cycle .', 'Question : What is GitHub and why is it used ?': 'Answer : GitHub is a platform for hosting our project codes , packages , methods where community will see and send their review . we also upload our project repository here for review and suggestion .', 'Question : Two resistances are connected in two gaps of Meter Bridge . The balance is 20cm from the zero end . A resistance of

In [116]:
output=response['Final_score']

In [117]:
output

'[\n{"Question":"Explain the process of Water cycle ?",\n "Type":"Theory",\n "Response":"Attempted",\n "Syntactic Score":50,\n "Semantic Score":60,\n "Contextual Score":40,\n "FirstStep Score":null,\n "Middle Step Score":null,\n "Last Step Score":null,\n "Total Score":"50"},\n\n{"Question":"What is GitHub and why is it used ?",\n "Type":"Theory",\n "Response":"Attempted",\n "Syntactic Score":70,\n "Semantic Score":80,\n "Contextual Score":60,\n "FirstStep Score":null,\n "Middle Step Score":null,\n "Last Step Score":null,\n "Total Score":"70"},\n\n{"Question":"Two resistances are connected in two gaps of Meter Bridge . The balance is 20cm from the zero end . A resistance of 15 ohms is connected in series with the smaller of the two . The null point shifts to 40cm . What is the value of the bigger resistance ?",\n "Type":"Theory",\n "Response":"Attempted",\n "Syntactic Score":80,\n "Semantic Score":90,\n "Contextual Score":70,\n "FirstStep Score":null,\n "Middle Step Score":null,\n "Last

In [118]:
data = json.loads(output)

In [119]:
df = pd.DataFrame(data)

In [120]:
df

,Question,Type,Response,Syntactic Score,Semantic Score,Contextual Score,FirstStep Score,Middle Step Score,Last Step Score,Total Score
0,Explain the process of Water cycle ?,Theory,Attempted,50.0,60.0,40.0,NaN,NaN,NaN,50
1,What is GitHub and why is it used ?,Theory,Attempted,70.0,80.0,60.0,NaN,NaN,NaN,70
2,Two resistances are connected in two gaps of M...,Theory,Attempted,80.0,90.0,70.0,NaN,NaN,NaN,80
3,Find the roots of the equation y ^ 3 23y ^ 2 +...,Numerical,Attempted,NaN,NaN,NaN,80.0,70.0,60.0,70
4,The resistivity of semiconductors and insulato...,Theory,Not Attempted,NaN,NaN,NaN,NaN,NaN,NaN,0


In [121]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:4266
Prompt Tokens:3327
Completion Tokens:939
Total Cost:0.0068685
